# IMPORTS

In [47]:
import numpy as np
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking, GRU, Dropout
from tensorflow.keras.callbacks import EarlyStopping #,TensorBoard

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Datos Necesarios para los Modelos

- Definicion de: Path/Ruta de alamcenamiento, Lista de Palabras, Cantidad de Personas, Cantidad de Videos por Persona y numero maximo de Frames.
- Armado de DICCIONARIO que mapea VALOR(seña/palabra) a -> indice/número
- Se almacenan en 2 variables los datos con los que se entrenaran los modelos "sequences" almacena en un arreglo los datos de cada video, "labels" es un arreglo de 0s y 1s que indica la clase a la que pertenece cada video. Esta celda puede ser reemplazada por la del apartado Limpieza de Datos (al final), el procesamiento es el mismo pero se tiene en cuenta "frames sucios".

In [42]:
STORE_PATH2 = os.path.join('LSA64_data')

# Lista de Palabras con las que se armarán las oraciones, no se encuentran las 64 palabras
signs_list = ['opaco', 'rojo', 'verde', 'amarillo', 'brillante', 'celeste', 'colores', 'rosa', 'mujer', 'enemigo', 'hijo', 'hombre', 'lejos','cajón','aprender','llamar','espumadera','amargo','dulce','leche','agua','comida','Argentina','Uruguay','pais','donde','apellido','burla','cumpleanos','desayuno','foto','hambre','mapa','moneda','musica','nave espacial','ninguno','nombre','paciencia','perfume','sordo','trampa','arroz','asado','caramelo','chicle','fideos','yogurt','aceptar','agradecer','apagar','aparecer','aterrizar','atrapar','ayudar','bailar','bañarse','comprar','copiar','correr','darse cuenta','dar','encontrar']
# signs_list = ['nacer','comida','brillante', 'mujer', 'hijo', 'hombre', 'lejos', 'aprender', 'espumadera','amargo','leche','Uruguay','pais','donde','ninguno','nombre','perfume','sordo','comprar','encontrar', 'nave espacial']
numberOfPersons = 10
numberOfVideosPerPerson = 5

maximumNumberOfFrames = 201

In [11]:
# DICCIONARIO que mapea VALOR(seña/palabra) a -> indice/número
sign_labels = {label : index for index, label in enumerate(signs_list)}
print(sign_labels)

{'nacer': 0, 'comida': 1, 'brillante': 2, 'mujer': 3, 'hijo': 4, 'hombre': 5, 'lejos': 6, 'aprender': 7, 'espumadera': 8, 'amargo': 9, 'leche': 10, 'Uruguay': 11, 'pais': 12, 'donde': 13, 'ninguno': 14, 'nombre': 15, 'perfume': 16, 'sordo': 17, 'comprar': 18, 'encontrar': 19, 'nave espacial': 20}


In [12]:
# LA EJECUCION DE ESTA CELDA PUEDE SER REEMPLAZADA POR LA DEL APARTADO LIMPIEZA DE DATOS
sequences, labels = [],[]
for sign in range(len(signs_list)):
    for personNumber in range(numberOfPersons-1):
        for videoNumber in range(numberOfVideosPerPerson):
            result = np.load(os.path.join(STORE_PATH2, signs_list[sign], f'''0{str(personNumber)}_0{str(videoNumber)}.npy'''))
            resultWithoutFace = []
# salto de a 1 frame para simular un "muestreo" de cada video y no usar todos los frames, no es un muestreo probabilistico
            for index in range(0,201,2):
#                 NO TOMO EN CUENTA LOS DATOS DE LA CARA
                resultWithoutFace.append(result[index][936:])
            sequences.append(resultWithoutFace)
            labels.append(sign_labels[signs_list[sign]])
            
sequences = np.array(sequences)
labels = to_categorical(labels).astype(int)

# DEFINICIÓN DE MODELOS

In [ ]:
model = Sequential()
model.add(Masking(mask_value=paddingValue, input_shape=(201, 1086)))
model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(40, 1086)))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(np.array(signs_list).shape[0], activation='softmax'))

In [ ]:
# CUARTO MODELO USADO GRU - GRU
paddingValue = 3
model = Sequential()
model.add(Masking(mask_value=paddingValue, input_shape=(201, 1086)))
model.add(GRU(128, return_sequences=True, activation='tanh'))
model.add(GRU(128, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(np.array(signs_list).shape[0], activation='softmax'))

In [ ]:
# QUINTO MODELO USADO
paddingValue = 3
model = Sequential()
model.add(Masking(mask_value=paddingValue, input_shape=(201, 1086)))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(128, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(np.array(signs_list).shape[0], activation='softmax'))

In [43]:
paddingValue = 3
model = Sequential()
model.add(Masking(mask_value=paddingValue, input_shape=(201, 150)))
model.add(GRU(128, return_sequences=True, activation='tanh'))
model.add(GRU(128, return_sequences=True, activation='tanh'))
model.add(GRU(64, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(np.array(signs_list).shape[0], activation='softmax'))

In [ ]:
# 30/11 pruebo para ver si con LSTM tambien demora 5 segundos sin datos de cara
paddingValue = 3
model = Sequential()
model.add(Masking(mask_value=paddingValue, input_shape=(201, 150)))
model.add(Dropout(0.4))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(Dropout(0.4))
model.add(LSTM(128, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(np.array(signs_list).shape[0], activation='softmax'))

In [28]:
# Armo un modelo para entrenar habiendo armado los arreglos con un salto de frames (sampleo o muestreo)
paddingValue = 3
model = Sequential()
model.add(Masking(mask_value=paddingValue, input_shape=(101, 150)))
model.add(Dropout(0.4))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(Dropout(0.4))
model.add(LSTM(128, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(np.array(signs_list).shape[0], activation='softmax'))

In [ ]:
model.summary()

# COMPILACIÓN DEL MODELO

In [44]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
esLoss = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=30, min_delta=0)

In [29]:
xTrain, xTest, yTrain, yTest = train_test_split(sequences, labels , test_size=0.05, random_state=1)

model.fit(xTrain, yTrain, validation_data=(xTest,yTest), epochs=500, callbacks=[esLoss])

# EVALUACION DEL MODELO

- Se evalua el modelo entrenado sobre todos los videos
- Se evalua el modelo sobre los datos que se separaron para Testeo (xTest, yTest)

In [ ]:
# evaluo las predicciones del modelo sobre TODO el set de datos de los videos procesados
threshold08 = 0.8
threshold09 = 0.9

totalProcessed = 0

predictedOkay08 = 0
predictedOkay09 = 0

for sign in range(len(signs_list)):
    for personNumber in range(numberOfPersons):
        for videoNumber in range(numberOfVideosPerPerson):
            videoData = np.load(os.path.join(STORE_PATH2, signs_list[sign], f'''0{str(personNumber)}_0{str(videoNumber)}.npy'''))
            # resultWithDistances = []
            # for index in range(201):
            #   resultWithDistances.append(getEuclideanDistances(videoData[index]))
            # print(f'''VIDEO: LSA64_data/{signs_list[sign]}/0{personNumber}_0{videoNumber}''')
            # videoData2 = []
            # for index in range(201):
                # if (index % 2 != 0):
                    # videoData2.append(videoData[index])
            resultWithoutFace = []
            for index in range(0,201,2):
                resultWithoutFace.append(videoData[index][936:])
#                 resultWithoutFace.append(videoData[index])
            response = model.predict(np.expand_dims(resultWithoutFace, axis=0))
            if (np.max(response) > threshold08):
#                 print('Seña leída: '+ signs_list[sign] + '------  Predice: ' + signs_list[np.argmax(response)])
                if (signs_list[sign] == signs_list[np.argmax(response)]):
                    predictedOkay08 = predictedOkay08 + 1
                    if (np.max(response) > threshold09):
                        predictedOkay09 = predictedOkay09 + 1
            totalProcessed = totalProcessed + 1
            
print(totalProcessed)
print(predictedOkay08)
print(str((predictedOkay08/totalProcessed)*100)+'%')
print(predictedOkay09)
print(str((predictedOkay09/totalProcessed)*100)+'%')

In [ ]:
j = 0
i = 0
list_of_wrongs = []
for index in range(len(xTest)):
    res = model.predict(np.expand_dims(xTest[index], axis=0))
    predicted = signs_list[np.argmax(res[0])]
    expected_result = signs_list[np.argmax(yTest[index])]
#     print(predicted)
#     print(expected_result)
    j += 1
    if (predicted == expected_result):
        i+=1
    else:
        list_of_wrongs.append(predicted + expected_result)
print(f"total:{j}, ok: {i}, mal {j - i}")

# GUARDA/CARGA DE MODELO
En caso de que el modelo funcione de manera correcta se pueden almacenar los pesos, sino tambien se pueden cargar (descomentar SAVE para guardar, de lo contrario descomentar LOAD para cargar)


In [45]:
# model.save('modelWeightsLSA64_500_1t.h5')
model.load_weights('modelos_pesos/GRUx3-GPUmia.h5')

# PRUEBA EN TIEMPO REAL DEL MODELO

Primer celda contiene funciones necesarias para mostrar puntos en imagen, deteccion de persona con MediaPipe (ya definidas y usadas en OBtencion de Datos.ipynb)

In [50]:
import cv2
import mediapipe

mp_holistic = mediapipe.solutions.holistic
mp_drawing = mediapipe.solutions.drawing_utils

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)                 #prediction from a frame
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

leftHandStyles = [0,138,255,2,1] #naranja
rightHandStyles = [231,217,0,2,1] #celeste
faceStyles = [80,110,10,0,1]
poseStyles = [70,100,5,2,1]
def draw_styled_landmarks(image, results):
    #     FACE
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        mp_drawing.DrawingSpec(color=(faceStyles[0],faceStyles[1],faceStyles[2]), thickness=faceStyles[3], circle_radius=faceStyles[4])
    )
    #     POSE/BODY
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(poseStyles[0],poseStyles[1],poseStyles[2]), thickness=poseStyles[3], circle_radius=poseStyles[4])
    )
    #     LEFT HAND
    mp_drawing.draw_landmarks(
        image,
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(leftHandStyles[0],leftHandStyles[1],leftHandStyles[2]), thickness=leftHandStyles[3], circle_radius=leftHandStyles[4])
    )
    #     RIGHT HAND
    mp_drawing.draw_landmarks(
        image,
        results.right_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(rightHandStyles[0],rightHandStyles[1],rightHandStyles[2]), thickness=rightHandStyles[3], circle_radius=rightHandStyles[4])
    )

def array_from_landmarks(results):
    # LEFT HAND array
    # len(results.left_hand_landmarks.landmark) = 21 landmarks for each hand, with 2 coordinates each landmark
    if (results.left_hand_landmarks):
        leftHandLandmarks = np.array([[result.x, result.y] for result in results.left_hand_landmarks.landmark]).flatten()
    else:
        leftHandLandmarks = np.zeros(21*2)


    # RIGHT HAND array
    # len(results.right_hand_landmarks.landmark) -> same for right hand
    if (results.right_hand_landmarks):
        rightHandLandmarks = np.array([[result.x, result.y ] for result in results.right_hand_landmarks.landmark]).flatten()
    else:
        rightHandLandmarks = np.zeros(21*2)


    # POSE array
    # len(results.pose_landmarks.landmark) -> 33 landmarks of 2 coordinates each one (X, Y)
    if (results.pose_landmarks):
        poseLandmarks = np.array([[result.x, result.y] for result in results.pose_landmarks.landmark]).flatten()
    else:
        poseLandmarks = np.zeros(33*2)


    # FACE array
    # len(results.face_landmarks.landmark) -> 468 landmarks of 2 coordinates each one
    if (results.face_landmarks):
        faceLandmarks = np.array([[result.x, result.y] for result in results.face_landmarks.landmark]).flatten()
    else:
        faceLandmarks = np.zeros(468*2)
        
    return np.concatenate((faceLandmarks, poseLandmarks, leftHandLandmarks, rightHandLandmarks))

In [53]:
sequence = [] #store 40 frames like the framesPerVideo
sentence = [] #store a list of 'words'
threshold = 0.7


caption = cv2.VideoCapture(0)
caption.set(cv2.CAP_PROP_FPS, 60)
i = 0
with mp_holistic.Holistic(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as holisticModel:
    while caption.isOpened():
        i = i + 1
        ret, frame = caption.read()
        
#         PARA REDIMENSIONAR IMAGEN
#         ret, normalFrame = caption.read()
#         frame = cv2.resize(normalFrame, (1920, 1080))
        image, results = mediapipe_detection(frame, holisticModel)
        draw_styled_landmarks(image, results)
        
        keypoints = array_from_landmarks(results)
        sequence.append(keypoints[936:]) #le quito datos de la cara
#         sequence.append(keypoints)
        
#         if len(sequence) == maximumNumberOfFrames:
        res = model.predict(np.expand_dims(sequence, axis=0))
        res = res[0]

        if res[np.argmax(res)] > threshold:
#           Si predice igual a ultima palabra predicha no se coloca
            if len(sentence) > 0:
                if signs_list[np.argmax(res)] != sentence[-1]:
                    sentence.append(signs_list[np.argmax(res)])
            else:
                sentence.append(signs_list[np.argmax(res)])
#         if len(sentence) > 5:
#             sentence = sentence[-5:]

        cv2.putText(image, ' '.join(sentence), (10,20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255,255,255), 2)
        cv2.putText(image, 'Frame °: ' + str(i), (10,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255,255,255), 2)
        cv2.imshow('Predicting sign..', image)
    
        if cv2.waitKey(10) & 0xFF == ord('q'): #press 'q' to break
            break
    caption.release()
    cv2.destroyAllWindows()

# LIMPIEZA DE LOS DATOS GUARDADOS

En esta seccion defino una funcion que salte los frames que representan datos "limpios" teniendo en cuenta en este caso como datos limpios a:
- Frames que no han detectado como MINIMO 1 mano, cara y posicion. Si falta alguna de estas 3 componentes el frame no se toma en cuenta.

- Se debe actualizar el valor de maximumNumberOfFrames segun el caso que se desee probar.

In [55]:
def isFrameClean(frame):
    if ((frame[0] != 0.0 and frame[1] != 0.0 and frame[934] != 0.0 and frame[935] != 0.0) and 
        (frame[958] != 0.0 and frame[959] != 0.0 and frame[960] != 0.0 and frame[961] != 0.0) and 
        ((frame[1018] != 0.0 and frame[1019] != 0.0 and frame[1003] != 0.0 and frame[1004] != 0.0) or 
        (frame[1044] != 0.0 and frame[1045] != 0.0 and frame[1084] != 0.0 and frame[1085] != 0.0)) 
        ):
        return True
    return False


paddingValue = 3

def paddData(sequence):
    paddLength = maximumNumberOfFrames - len(sequence)
    return np.pad(sequence, [(0,paddLength),(0,0)], mode='constant', constant_values=paddingValue)

In [ ]:
# CELDA QUE PERMITE EL REEMPLAZO NOMBRADO ARRIBA DEL PROYECTO
sequences, labels = [],[]
for sign in range(len(signs_list)):
    for personNumber in range(numberOfPersons-1):
        for videoNumber in range(numberOfVideosPerPerson):
            result = np.load(os.path.join(STORE_PATH2, signs_list[sign], f'''0{str(personNumber)}_0{str(videoNumber)}.npy'''))
            resultWithFace = []

#             el 201 es la longitud de frames con la que se almacenaron todos los videos

            for index in range(0,201,2):#me salto de a 1 frame para probar con muestreo de cada video y no usar todos los frames, no es un muestreo probabilistico
#                 resultWithoutFace.append(result[index][936:])
                resultWithFace.append(result[index])
#             al haberme salteado de a 2 frames, de los 201 me quedan 67, 
#             entonces ahora limpio los frames "sucios" y paddeo con valor 3 hasta completar los 67
#             despues debería probar que en lugar de completar con valor de padding 3, usar el 
#             frame previo o siguiente al que se quitó.

            cleanedResults = []
            for frame in range(len(resultWithFace)):
#                 En caso de frame "sucio" no debería reemplazarlo por dato de padding, por que el padding debería 
#                 ir solo al final, por lo que tomo un frame de izquierda o derecha.
#                 Sino no reemplazo con nada y hago el padding al final como está ahora
                if (isFrameClean(resultWithFace[frame])):
                    cleanedResults.append(resultWithFace[frame])
            if (len(cleanedResults) != maximumNumberOfFrames):
                cleanedResults = paddData(cleanedResults)
            sequences.append(cleanedResults)
            labels.append(sign_labels[signs_list[sign]])
sequences = np.array(sequences)
labels = to_categorical(labels).astype(int)